In [5]:
import imports
import fea, split, weights, model, train

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from Bio import SeqIO

fasta_file = 'input/main/dna_10.fasta'  
sequences = []
metadata = []

for record in SeqIO.parse(fasta_file, "fasta"):
    metadata.append(record.description)  
    seq = str(record.seq).upper()
    sequences.append(seq)
labels = np.ones(len(sequences))
print(metadata[0])
print(labels)
print(sequences[0])

chromosome:GRCh38:19:10144450:10145050:-1
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
TTGTATCATAGCAACAACCCCAAATCACAGTGACTTAAAACCACAGAAGCTGGACTGGGTGCGGTGGCTCATGCCTGTCAGGAGTTCAAGACCAAGACCAGCCTGGCCGACATACCTGTCTCTACTAAAAATAAATTAGCCAGGGTTAGCTGGGTGTGGTGGTGGGCACCTGTAATCCCAGCTACTTGGGAGGCCGAGGCACGAGAATCGCTTGAACCTGGGAGGAGGAGTTGCAGTGAGCCGAGATTGTGCCACTGCACTCTAACCTGGGTGACAGAGTGAAACTCTCTCAACAAAAACCACGGTAGCCGCTGTCTCGCTCATACGAGGTGCCCATCACAGGTCAGCAGAGGACTCCACCTGTTGTGTCCCTTGGATATCTGGGCCGCTGGAGTCAGAAGTTGCCAGTGTTTGAGCCCAAGGGAAAGCACACAGGACATTCCCCAGGCAGTGACATCCTCTTTGGTCACTTCTGCCCACGTCACATGCCAGGAAACACAAGATGCCAAGTAGTCGGGAGGCAGGAGCCTGGGGTATTTGAGCAGTGTCAGCCACCACCGTAGCCAGTAATGCGCAGAGGGTGAATGCGTGCTTCAAACTG


In [6]:
merscore1, merscore2, merscore3, merscore4, merscore5, merscore6,kmer1, kmer2, kmer3 = fea.generate_features(sequences)

merscore1
merscore2
merscore3
merscore4
merscore5
merscore6


In [7]:
import numpy as np
feat = np.concatenate(
            (
                merscore1, merscore2, merscore3,
                merscore4, merscore5, merscore6,
                kmer1, kmer2, kmer3
            ),
            axis=1
        )
print(feat[0])

[ 0.01064354  0.00281239  0.00070923 -0.0023496  -0.03285118 -0.09573726
  0.2562396   0.25956739  0.281198    0.20299501  0.04658902  0.06655574
  0.08985025  0.03494176  0.09650582  0.06156406  0.0266223   0.06322795
  0.06322795  0.06821963  0.05990017  0.06821963  0.03161398  0.0515807
  0.08485857  0.02995008  0.01497504  0.01663894  0.01331115  0.01164725
  0.01996672  0.01996672  0.00665557  0.01663894  0.01663894  0.0249584
  0.0266223   0.02163062  0.00831947  0.01164725  0.00831947  0.00665557
  0.01830283  0.0266223   0.03660566  0.01331115  0.03161398  0.01164725
  0.00998336  0.01663894  0.00831947  0.00831947  0.00499168  0.00499168
  0.00665557  0.01497504  0.02828619  0.01164725  0.01331115  0.01497504
  0.02995008  0.00499168  0.01830283  0.02995008  0.00499168  0.01497504
  0.02329451  0.01331115  0.01996672  0.02163062  0.01164725  0.01663894
  0.02828619  0.00998336  0.00998336  0.00665557  0.00998336  0.00499168
  0.02329451  0.00831947  0.00499168  0.01164725  0.0

In [8]:
print(feat.shape)
print(labels.shape)

(10, 90)
(10,)


In [9]:
Xt = np.expand_dims(feat, axis=2).astype(np.float32)
yt = labels.astype(np.float32)

print(Xt.shape)  # (10, 90, 1)
print(yt.shape)  # (10,)

(10, 90, 1)
(10,)


In [11]:
from tensorflow.keras.models import load_model
model = load_model("output/dna.keras") 

In [12]:
print(Xt.shape)
predictions = model.predict(Xt)  # (samples, timesteps, features)
predictions = predictions.flatten()  

(10, 90, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step


In [13]:
print(predictions)
print(predictions.shape) 

[0.691854   0.68687344 0.9028571  0.99998236 0.8948073  0.54692966
 0.9414488  0.5723682  0.8222191  0.6033875 ]
(10,)


In [14]:
import pandas as pd

df = pd.DataFrame({
    "metadata": metadata,
    "prediction": predictions
})

In [15]:
df.to_csv("output/dna_10.csv", index=False)
print("Predictions saved to dna_10.csv")

Predictions saved to dna_10.csv
